In [60]:
import tensorflow as tf
import pandas as pd

In [65]:
csv_label_name = "medv" # value of the house \$1000s
csv_column_names = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
feature_names = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]

train_data = pd.read_csv("datasets/boston-housing/train.csv", names=csv_column_names, skiprows=1, skipinitialspace=True)
test_data = pd.read_csv("datasets/boston-housing/test.csv", names=csv_column_names, skiprows=1, skipinitialspace=True)

In [66]:
feature_columns = [
    tf.feature_column.numeric_column(feature) for feature in feature_names
]
    
estimator = tf.estimator.DNNRegressor(hidden_units=[10, 10], feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1267c9350>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfW4DQB', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


In [69]:
def input_fn(dataset, shuffle=0, batch=128):
    data_set = tf.data.Dataset.from_tensor_slices((dict(dataset), dataset[csv_label_name].values))
    if shuffle > 0:
        data_set = data_set.shuffle(shuffle)
    return (data_set.batch(128)
        # Repeat forever
        .repeat().make_one_shot_iterator().get_next())

def input_fn_train():
    return input_fn(train_data, shuffle=1000)

def input_fn_test():
    return input_fn(test_data)

estimator.train(input_fn=input_fn_train, steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfW4DQB/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5001 into /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfW4DQB/model.ckpt.
INFO:tensorflow:loss = 2647.733, step = 5001
INFO:tensorflow:global_step/sec: 237.866
INFO:tensorflow:loss = 3398.3218, step = 5101 (0.422 sec)
INFO:tensorflow:global_step/sec: 385.239
INFO:tensorflow:loss = 2811.7888, step = 5201 (0.261 sec)
INFO:tensorflow:global_step/sec: 250.924
INFO:tensorflow:loss = 4297.0015, step = 5301 (0.398 sec)
INFO:tensorflow:global_step/sec: 332.12
INFO:tensorflow:loss = 3905.2136, step = 5401 (0.301 sec)
INFO:tensorflow:global_step/sec: 310.563
INFO:tensorflow:loss = 1948.685, step = 5501 

In [ ]:
evaluation = estimator.evaluate(input_fn=input_fn_test)
loss_score = evaluation["loss"]
print("Loss: {0:f}".format(loss_score))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-23-22:12:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpfW4DQB/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
test_data = pd.read_csv("datasets/boston-housing/test.csv", names=csv_column_names, skiprows=1, skipinitialspace=True)
predictions = estimator.predict(input_fn=input_fn(test_data))
list(predictions)